**DATA COLLECTION**

In [2]:
#Importing required libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit

In [3]:
def beautifulsoup_func(url):
    requests_object = requests.get(url)
    source_code = requests_object.text
    soup = BeautifulSoup(source_code,'html.parser')
    return soup

def get_tableheader():
    table_header = soup.find_all('th')
    header = [h.text for h in table_header]
    return header

def df_to_csv(dataframe,filename):
    return dataframe.to_csv(filename)

In [4]:
# Scraping Match Summary Table

url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=2022%2F23;trophy=89;type=season'

soup = beautifulsoup_func(url)  # requesting and parsing url with requests and beautifulsoup

header = get_tableheader()    # getting table headers

table_row = soup.find_all('tr',class_='data1')  # parsing table rows

summary_df = pd.DataFrame(columns = header)  #vreating empty dataframe with columns as table headers

# Iterating through table data and adding it into dataframe

for row in table_row:
    data = row.find_all('td')
    row_data = [d.text for d in data]
    href = [str(link.get('href')) for link in soup.find_all('a',class_ = 'data-link') if 'match' in str(link.get('href'))]
    length = len(summary_df)
    summary_df.loc[length] = row_data
    
df_to_csv(summary_df,'T20_worldcup_2022_match_summary')

In [ ]:
# Scraping summary of individual batsman and bowlers and what team they belongs to respectively.

url = 'https://stats.espncricinfo.com' + href[0]

soup = beautifulsoup_func(url)

header = get_tableheader()

Batting_header=header[:8]

Batting_header.insert(0,'batting_position')          # adding batting position of the batsman

batting_df = pd.DataFrame(columns = Batting_header)

Bowling_header=header[8:19]

Bowling_header.insert(0,'S.NO.')                     

bowling_df = pd.DataFrame(columns = Bowling_header)

temp_df = pd.DataFrame(columns=['Team 1','Team 2']) # creating temp_df for players for what team they belong to

for url in href:
    
    soup = beautifulsoup_func('https://stats.espncricinfo.com' + url)
    
    tr = soup.find_all('tr')   
    
    temp_nestedlist = [] 
    tlist = []
    acc = 0
    l = len(temp_df)
    
    team_name = soup.find_all('span',class_='ds-text-title-xs ds-font-bold ds-text-typo') # scraping team name 
        
    for i in team_name:                   #Iterating team names with accordance to batting first        
        k = [t.text for t in i]
        tlist.append(k[0])              
        acc+=1

    if len(tlist)==0:                     # len(tlist)==0 means that the match is abondoned
        continue
    temp_df.loc[l] = tlist                # temp_df contains Team1 and Team2 where Team1 has batted first

    for r in tr:                          #Iterating through table rows
        count = 1 
        count1 = 1
        data = r.find_all('td')
        l =[d.text for d in data]
        temp_nestedlist.append(l)
        
    for list in temp_nestedlist:       # Only accepting table rows of batting summary and bowling summary
        if len(list)==8:
            count1=1
            list.insert(0,count)          # Inserting batting positon
            length=len(batting_df)
            batting_df.loc[length] = list
            count+=1
        elif(len(list)==11):
            count=1
            list.insert(0,count1)
            length1=len(bowling_df)       
            bowling_df.loc[length1] = list
            count1+=1
        else:
            #continue
            pass


In [ ]:
batting_df['Country'] = ''

bowling_df['Country'] = ''

c1 = 0
c = 1
c2 = 0

c3=0
c4=1
c5=0

for i in range(0,len(batting_df)-1):
    if c ==1:
        if (batting_df['batting_position'][i] < batting_df['batting_position'][i+1]):
            batting_df['Country'][i] = temp_df['Team 1'][c1]
        else:
            batting_df['Country'][i] = temp_df['Team 1'][c1]
            c+=1
            c1+=1
    elif c == 2:
        if (batting_df['batting_position'][i]< batting_df['batting_position'][i+1]):
            batting_df['Country'][i] = temp_df['Team 2'][c2]
        else:
            batting_df['Country'][i] = temp_df['Team 2'][c2]                                                 
            c=1
            c2+=1
            

for i in range(0,len(bowling_df)-1):
    if c4 ==1:
        if (bowling_df['S.NO.'][i] < bowling_df['S.NO.'][i+1]):
            bowling_df['Country'][i] = temp_df['Team 2'][c3]
        else:
            bowling_df['Country'][i] = temp_df['Team 2'][c3]
            c4+=1
            c3+=1
    elif c4 == 2:
        if (bowling_df['S.NO.'][i]< bowling_df['S.NO.'][i+1]):
            bowling_df['Country'][i] = temp_df['Team 1'][c5]
        else:
            bowling_df['Country'][i] = temp_df['Team 1'][c5]                                                 
            c4=1
            c5+=1
                        

batting_df.rename(columns = {'\xa0':'Dismissal','BATTING':'Batter','R':'Runs','B':'Balls','M':'Minutes'},inplace = True)


bowling_df.rename(columns = {'BOWLING':'Bowler','O':'Overs','R':'Runs','M':'Maidens','W':'Wickets'},inplace = True)


**DATA CLEANING**

In [ ]:

# Replacing 

batting_df['Batter'] = batting_df['Batter'].str.replace('\xa0','')

batting_df['Batter'] = batting_df['Batter'].str.replace('†','(WK)')

batting_df['Dismissal'] = batting_df['Dismissal'].str.replace('\xa0','')

batting_df['Dismissal'] = batting_df['Dismissal'].str.replace('†','(WK)')

batting_df.Country.loc[len(batting_df['Country'])-1] = batting_df.Country.loc[len(batting_df['Country'])-2]

# cleaning bowling data

bowling_df['Bowler'] = bowling_df['Bowler'].str.replace('\xa0','')

bowling_df.Country.loc[len(bowling_df['Country'])-1] = bowling_df.Country.loc[len(bowling_df['Country'])-2]


In [ ]:
df_to_csv(batting_df,'batting_summary_T20worldcup2022.csv')

df_to_csv(bowling_df,'bowling_summary_T20worldcup2022.csv')
            
batting_summary = pd.read_csv('batting_summary_T20worldcup2022.csv')

bowling_summary = pd.read_csv('bowling_summary_T20worldcup2022.csv')

batting_summary = batting_summary.drop("Unnamed: 0",axis=1)

bowling_summary = bowling_summary.drop("Unnamed: 0",axis=1)